In [2]:
import pandas as pd
import numpy as np

ctd=pd.read_csv('..\input_data\cdc_to_dc.csv')
ctd


,From,To,Distance,Duration
0,nei-lu-CDC,cheng-du-DC,1841,1279
1,nei-lu-CDC,dong-guan-DC,2131,1386
2,nei-lu-CDC,guang-zhou-DC,2122,1391
3,nei-lu-CDC,nei-lu-CDC,0,0
4,nei-lu-CDC,nan-jing-DC,999,738
5,nei-lu-CDC,qing-dao-DC,651,491
6,nei-lu-CDC,gang-kou-CDC,2018,1371
7,nei-lu-CDC,shang-hai-DC,1154,885
8,nei-lu-CDC,shen-zhen-DC,2147,1398
9,nei-lu-CDC,tian-jin-DC,91,115


距离和时效矩阵

In [3]:
#matrix['UniqueID'] = matrix.groupby(['From', 'To']).cumcount() + 1

distance_matrix_ctd = ctd.pivot_table(index='From', columns='To', values='Distance')
distance_matrix_ctd.drop('gang-kou-CDC',axis=1,inplace=True)
distance_matrix_ctd.drop('nei-lu-CDC',axis=1,inplace=True)
distance_matrix_ctd = distance_matrix_ctd.reset_index()
distance_matrix_ctd.to_csv('../input_data/d_matrix_ctd.csv')


In [4]:
duration_matrix_ctd = ctd.pivot_table(index='From', columns='To', values='Duration')
duration_matrix_ctd.drop('gang-kou-CDC',axis=1,inplace=True)
duration_matrix_ctd.drop('nei-lu-CDC',axis=1,inplace=True)
duration_matrix_ctd = duration_matrix_ctd.reset_index()
duration_matrix_ctd.to_csv('../input_data/t_matrix_ctd.csv')


In [5]:
dts=pd.read_csv('../input_data/dc_to_st.csv')
dts

,From,To,Distance,Duration
0,cheng-du-DC,bo-zhou-shi,1461,1047
1,cheng-du-DC,chu-zhou-shi,1657,1122
2,cheng-du-DC,fu-yang-shi,1442,1019
3,cheng-du-DC,he-fei-shi,1515,1023
4,cheng-du-DC,lu-an-shi,1437,984
...,...,...,...,...
2115,chong-qing-DC,ning-bo-shi,1810,1260
2116,chong-qing-DC,shao-xing-shi,1684,1232
2117,chong-qing-DC,tai-zhou-shi,1765,1290
2118,chong-qing-DC,wen-zhou-shi,1739,1255


In [6]:

dts['UniqueID'] = dts.groupby(['From', 'To']).cumcount() + 1
distance_matrix_dts = dts.pivot_table(index='From', columns=['To','UniqueID'], values='Distance')
distance_matrix_dts = distance_matrix_dts.reset_index()
distance_matrix_dts.to_csv('../input_data/d_matrix_dts.csv')


In [7]:

t_matrix_dts = dts.pivot_table(index='From', columns=['To','UniqueID'], values='Duration')
t_matrix_dts = t_matrix_dts.reset_index()
t_matrix_dts.to_csv('../input_data/t_matrix_dts.csv')

供应关系矩阵

In [8]:
#仓库出发
from_dc=np.array(distance_matrix_dts['From'])
#到RCD
to_dc=np.array(distance_matrix_ctd.columns)
to_dc=np.delete(to_dc,0)
#CDC出发
from_cdc=np.array(distance_matrix_ctd['From'])
#到store
to_store=np.array(distance_matrix_dts.columns)
to_store=np.delete(to_store,0)
print("cdc:",from_cdc.size,"\n","dc:",from_dc.size,"\n","rdc",to_dc.size,"\n","store:",to_store.size)

cdc: 2 
 dc: 20 
 rdc 18 
 store: 106


In [9]:
#ctd
M1=np.zeros((2,18)) #国产
M3=np.zeros((2,18)) #进口
#dts
M2=np.zeros((20,106)) #国产
M4=np.zeros((20,106)) #进口

仓库状态

In [10]:
O=np.zeros((20,1))  #是否开仓
Z=np.zeros((20,1))  #是否智能

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [27]:
dc_info=pd.read_csv('../input_data/df_proc.csv')
dc_info=dc_info.set_index('Name')
dc_info

,Capacity,Processing_fee,Opening_fee
Name,,,
cheng-du-DC,2000,0.00670,25
dong-guan-DC,1300,0.00700,25
guang-zhou-DC,2500,0.00750,25
nei-lu-CDC,10000,0.00650,0
nan-jing-DC,2500,0.00700,25
qing-dao-DC,1000,0.00700,25
gang-kou-CDC,8000,0.00750,0
shang-hai-DC,3500,0.00720,25
shen-zhen-DC,1800,0.00750,25


In [57]:
Pf=np.zeros((20,2),dtype=float) #处置成本
Of=np.zeros((20,1))  #开店成本
Cap=np.zeros((20,1))  #最大容量
tmp=0
for i in from_dc:
    a=dc_info.loc[i][1]
    Pf[tmp,0]=a
    Pf[tmp,1]=0.5*a
    Of[tmp]=dc_info.loc[i][2]
    Cap[tmp]=dc_info.loc[i][0]
    tmp+=1
#print(Pf)
#print(Of)
#print(Cap)


C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_22508\2846973989.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  a=dc_info.loc[i][1]
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_22508\2846973989.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Of[tmp]=dc_info.loc[i][2]
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_22508\2846973989.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Cap[tmp]=dc_info.loc[i][0]


门店对应订单

In [82]:
order_info_dm=pd.read_csv('..\input_data\df_order_dm.csv')
order_info_im=pd.read_csv('..\input_data\df_order_im.csv')
order_info_dm['UniqueID'] = order_info_dm.groupby(['Name','SKU']).cumcount() + 1
order_info_dm=order_info_dm.set_index(['Name','UniqueID'])
#print(order_info_dm)
order_info_im['UniqueID'] = order_info_im.groupby(['Name','SKU']).cumcount() + 1
order_info_im=order_info_im.set_index(['Name','UniqueID'])
#print(order_info_im)
Q_dm=np.zeros((106,1))
Q_im=np.zeros((106,1))
tmp=0
for i in to_store:
    Q_dm[tmp]=order_info_dm.loc[i][0]
    Q_im[tmp]=order_info_im.loc[i][0]
    tmp+=1
print(Q_dm[:5])
print(Q_im[:5])

    


[[ 37.6]
 [ 57.5]
 [ 19.1]
 [363.4]
 [ 23.3]]
[[ 15.3]
 [ 23.6]
 [ 11.1]
 [328.6]
 [  7.7]]


C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_22508\1754198815.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Q_dm[tmp]=order_info_dm.loc[i][0]
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_22508\1754198815.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Q_im[tmp]=order_info_im.loc[i][0]
